In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import re
%matplotlib inline
%load_ext blackcellmagic

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# parse vcf 

not the latest version, should check summarize_HTMCP_validation_strelka_mutations_20190624, compare parser result and results from validation summarization script for 3 samples. they matched. checked 20191210


20191210, to add allele frequency info from the vcf, can incorporate filtering from this nb filter_strelka_vcfs.ipynb 
##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Read depth for tier1">
##FORMAT=<ID=DP2,Number=1,Type=Integer,Description="Read depth for tier2">
##FORMAT=<ID=TAR,Number=2,Type=Integer,Description="Reads strongly supporting alternate allele for tiers 1,2">
##FORMAT=<ID=TIR,Number=2,Type=Integer,Description="Reads strongly supporting indel allele for tiers 1,2">
##FORMAT=<ID=TOR,Number=2,Type=Integer,Description="Other reads (weak support or insufficient indel breakpoint overlap) for tiers 1,2">
##FORMAT=<ID=DP50,Number=1,Type=Float,Description="Average tier1 read depth within 50 bases">
##FORMAT=<ID=FDP50,Number=1,Type=Float,Description="Average tier1 number of basecalls filtered from original read depth within 50 bases">
##FORMAT=<ID=SUBDP50,Number=1,Type=Float,Description="Average number of reads below tier1 mapping quality threshold aligned across sites within 50 bases">


new

In [94]:
# export
import numpy as np
import pandas as pd
import re
import os
from tqdm import tqdm


class VcfParser:
    def __init__(self, impacts, vcf=None, df=None):
        
        IMPACTS = ["HIGH", "MODERATE", "LOW", "MODIFIER"]
        impacts = [i.upper() for i in impacts]
        # make sure impact sorted from high to modifier
        impacts = [i for i in IMPACTS if i in impacts]
        self.vcf = vcf
        self.df = df
        self.impacts = impacts
        self.IMPACTS = IMPACTS
        self.BASES = np.array(['A', 'C', 'G', 'T'])
        print(f"parsing variants with impacts of: {self.impacts}!")

    def keep(self, series, patterns):
        return series.str.contains("|".join(patterns), case=False)

    def parse_effect(self, line):
        effs = line.split("EFF=")[1].split(",")
        effs = [
            list(np.array(re.split("\(|\|", ef))[[0,1,2,4,6,9]]) for ef in effs if (
                any([''.join(['(', i, '|']) in ef for i in self.IMPACTS]) 
            )
        ]
        for impact in [a.upper() for a in self.impacts]:
            pick = [e for e in effs if impact in e]
            if pick:
                break
        return pick
    
    def indel_coverage(self, s):
        cov = s.split(':')
        cov1 = cov[0]
        alt1 = cov[2].split(',')[0]
        indel1 = cov[3].split(',')[0]
        return [cov1, alt1]

    def parse_indel_coverage(self, df):
        df[['t_cov_tier1', 't_alt_tier1']] = (
            pd.DataFrame(df['tumour'].apply(
                lambda x: self.indel_coverage(x)).values.tolist(), index=df.index))
        df[['n_cov_tier1', 'n_alt_tier1']] = (
            pd.DataFrame(df['normal'].apply(
                lambda x: self.indel_coverage(x)).values.tolist(), index=df.index))
        return df
    
    def snv_coverage(self, s, alt):
        # DP:FDP:SDP:SUBDP:AU:CU:GU:TU
        cov = s.split(':')
        acgt = np.zeros(4)
        for i, c in enumerate(cov[4:]):
            acgt[i] = c.split(',')[0]
        return [acgt.sum(), acgt[np.asscalar(np.where(self.BASES == alt)[0])]]
    
    def parse_snv_coverage(self, df):
        
        df[['t_cov_tier1', 't_alt_tier1']] = (
            pd.DataFrame(df[['tumour', 'alt']].apply(
                lambda x: self.snv_coverage(x[0], x[1]), axis=1).values.tolist(), index=df.index))
        df[['n_cov_tier1', 'n_alt_tier1']] = (
            pd.DataFrame(df[['normal', 'alt']].apply(
                lambda x: self.snv_coverage(x[0], x[1]), axis=1).values.tolist(), index=df.index))
        return df

    def parse_vcf(self, vcf_type):
        if os.stat(self.vcf).st_size == 0:
            print(f"WARNING: empty file>{self.vcf}")
            return pd.DataFrame()
        print(f"parsing vcf: {self.vcf}")
        df = pd.read_csv(self.vcf, comment="#", sep="\t", header=None, low_memory=False)
        patient = self.vcf.split("/")[4]
        df = df[[0, 1, 3, 4, 7, 9, 10]]
        df.columns = ["chr", "pos", "ref", "alt", "effect", "normal", "tumour"]
        df = df[self.keep(df["effect"], self.impacts)]
        # sometimes the dp filed are nan
        df = df.dropna()

        if not df.empty:
            if vcf_type.lower() == 'indels':
                df = self.parse_indel_coverage(df)
            
            if vcf_type.lower() == 'snvs':
                df = self.parse_snv_coverage(df)
            df['anno'] = df['effect'].apply(lambda x: self.parse_effect(x))           
            df["patient"] = patient
            df = df.explode('anno')
            df[['impact','impact_type', 'type', 'AA_change', 'gene', 'transcript']] = (
                pd.DataFrame(df['anno'].values.tolist(), index=df.index)
            )
            
            df = df.drop(["effect", "anno", "tumour", "normal"], axis=1)
            return df
        return pd.DataFrame()

    def summarize_cohort(self):
        dfmg = pd.DataFrame()
        for ix, row in tqdm(self.df.iterrows()):
            patient = ix
            snv_vcf = row["snv"]
            indel_vcf = row["indel"]
            self.vcf = snv_vcf
            snv_df = self.parse_vcf('snvs')
            self.vcf = indel_vcf
            indel_df = self.parse_vcf('indels')
            mdf = pd.concat([snv_df, indel_df], sort=False)
            mdf = mdf.drop_duplicates(keep="first")
            if dfmg.empty:
                dfmg = mdf
            else:
                dfmg = pd.concat([dfmg, mdf], sort=False)
        return dfmg

In [95]:
# vcf = '/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02025-01A-01D/hg19a/SPC/B68907_B68872/strelka/65713/bwa/results/passed.somatic.snvs.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf'
impacts = ['high', 'moderate', 'low']
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/validation/validation_2nd_strelka_20191010.xls'
df = pd.read_excel(f, header=None)
df.columns = ['patient', 'tlib', 'nlib', 'snv', 'indel']
df.set_index('patient', inplace=True)
df.head(10)

,tlib,nlib,snv,indel
patient,,,,
HTMCP-03-06-02025,B68907,B68872,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
HTMCP-03-06-02044,B68919,B68884,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
HTMCP-03-06-02060,B68930,B68895,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
HTMCP-03-06-02061,B68941,B68906,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
HTMCP-03-06-02062,B68951,B68918,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
HTMCP-03-06-02066,B68874,B68929,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
HTMCP-03-06-02067,B68886,B68940,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
HTMCP-03-06-02072,B68897,B68950,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...
HTMCP-03-06-02081,B68920,B68885,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...,/projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-...


In [96]:
f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/124_patients_bam_vcf_20190115.csv'
df = pd.read_csv(f1, index_col='patient')
ndf = df[['strelka_snv_vcf', 'strelka_indel_vcf']].dropna()
ndf.columns = ['snv', 'indel']
# 118 patients
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients.txt'
with open(f) as file:
    patients = [line.strip() for line in file]
assert len(patients) == 118
ndf = ndf.reindex(patients)
ndf.head(2)
ndf.shape

(118, 2)

In [97]:
parser = VcfParser(impacts, df=ndf)
dfmg = parser.summarize_cohort()



0it [00:00, ?it/s]

parsing variants with impacts of: ['HIGH', 'MODERATE', 'LOW']!
parsing vcf: /projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02001/hg19a/GSH/A37234_A37159/strelka/59844/bwa/results/passed.somatic.snvs.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf




1it [00:00,  1.15it/s]

parsing vcf: /projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02001/hg19a/GSH/A37234_A37159/strelka/59844/bwa/results/passed.somatic.indels.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf
parsing vcf: /projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02002/hg19a/GSH/A37235_A37160/strelka/59848/bwa/results/passed.somatic.snvs.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf




2it [00:01,  1.33it/s]

...



116it [02:15,  2.07s/it]

parsing vcf: /projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02442/hg19a/GSH/A78437_A76750/strelka/60141/bwa/results/passed.somatic.snvs.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf
parsing vcf: /projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02442/hg19a/GSH/A78437_A76750/strelka/60141/bwa/results/passed.somatic.indels.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf




117it [02:16,  1.71s/it]

parsing vcf: /projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02447/hg19a/GSH/A78440_A76753/strelka/60142/bwa/results/passed.somatic.snvs.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf
parsing vcf: /projects/somatic/NCI_SAIC_HIV_Cervical/HTMCP-03-06-02447/hg19a/GSH/A78440_A76753/strelka/60142/bwa/results/passed.somatic.indels.eff.dbSNP_v149.cosmic_v82.clinvar_20170801.annotations.classic.vcf




118it [02:17,  1.17s/it]


In [98]:
dfmg[dfmg['ref']=='AG']#.unique()

,chr,pos,ref,alt,t_cov_tier1,t_alt_tier1,n_cov_tier1,n_alt_tier1,patient,impact,impact_type,type,AA_change,gene,transcript
175,2,232072964,AG,A,54,12,55,56,HTMCP-03-06-02001,FRAME_SHIFT+SPLICE_SITE_REGION,HIGH,,Q59,ARMC9,ENST00000349938
175,2,232072964,AG,A,54,12,55,56,HTMCP-03-06-02001,FRAME_SHIFT+SPLICE_SITE_REGION,HIGH,,Q59,ARMC9,ENST00000440107
272,6,136596998,AG,A,111,65,51,49,HTMCP-03-06-02002,FRAME_SHIFT,HIGH,,P555,BCLAF1,ENST00000531224
272,6,136596998,AG,A,111,65,51,49,HTMCP-03-06-02002,FRAME_SHIFT,HIGH,,P553,BCLAF1,ENST00000353331
272,6,136596998,AG,A,111,65,51,49,HTMCP-03-06-02002,FRAME_SHIFT,HIGH,,P555,BCLAF1,ENST00000527536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34318,22,19839660,AG,A,101,65,45,43,HTMCP-03-06-02435,FRAME_SHIFT,HIGH,,L42,C22orf29,ENST00000407472
34318,22,19839660,AG,A,101,65,45,43,HTMCP-03-06-02435,FRAME_SHIFT,HIGH,,L42,C22orf29,ENST00000405640
34318,22,19839660,AG,A,101,65,45,43,HTMCP-03-06-02435,FRAME_SHIFT,HIGH,,L42,C22orf29,ENST00000416337
34360,22,23973155,AG,A,124,77,61,61,HTMCP-03-06-02435,SPLICE_SITE_REGION+INTRON,LOW,,,C22orf43,ENST00000460169


In [100]:
of = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/118_patients_HML_variant_summary_with_AF_20191210.txt'
dfmg.to_csv(of, sep='\t', index=False)

In [101]:
pwd

'/projects/trans_scratch/validations/workspace/szong/cervical2/nbs'

In [144]:
def test(a,b):
    c = 'xxx'
    if a != b:
        c = 'mmm'
    return c

In [147]:
df[['tlib', 'nlib']].apply(lambda x: test(x[0], x[1]), axis=1)

patient
HTMCP-03-06-02025    mmm
HTMCP-03-06-02044    mmm
HTMCP-03-06-02060    mmm
HTMCP-03-06-02061    mmm
HTMCP-03-06-02062    mmm
                    ... 
HTMCP-03-06-02423    mmm
HTMCP-03-06-02425    mmm
HTMCP-03-06-02426    mmm
HTMCP-03-06-02439    mmm
HTMCP-03-06-02446    mmm
Length: 86, dtype: object

In [138]:
def parse_snv_coverage(s, alt):
    # DP:FDP:SDP:SUBDP:AU:CU:GU:TU
    cov = s.split(':')
    acgt = np.zeros(4)
    for i, c in enumerate(cov[4:]):
        acgt[i] = c.split(',')[0]
    return [acgt.sum(), acgt[np.asscalar(np.where(BASES == alt)[0])]]


In [85]:
dfmg.head()

,chr,pos,ref,alt,t_cov_tier1,t_alt_tier1,t_indel_tier1,n_cov_tier1,n_alt_tier1,n_indel_tier1,patient,impact,impact_type,type,AA_change,gene,transcript
0,1,121484536,C,T,58,0,0,114,0,0,HTMCP-03-06-02025-01A-01D,INTERGENIC,MODIFIER,,,,
1,1,121484671,T,G,75,0,0,127,0,0,HTMCP-03-06-02025-01A-01D,INTERGENIC,MODIFIER,,,,
2,1,121484700,T,A,73,0,0,122,0,0,HTMCP-03-06-02025-01A-01D,INTERGENIC,MODIFIER,,,,
3,1,121485137,G,C,202,0,0,518,0,0,HTMCP-03-06-02025-01A-01D,INTERGENIC,MODIFIER,,,,
4,2,92315416,T,C,26,0,0,47,0,0,HTMCP-03-06-02025-01A-01D,INTERGENIC,MODIFIER,,,,


In [64]:
df = pd.DataFrame({'A': [[1, 2, 3], 'foo', [], [5, 4]], 'B': [[11, 22], [], [], [99, 88]]})
df
df1 = df.explode('A')
df1
df1.explode('B').drop_duplicates()

,A,B
0,"[1, 2, 3]","[11, 22]"
1,foo,[]
2,[],[]
3,"[5, 4]","[99, 88]"


,A,B
0,1,"[11, 22]"
0,2,"[11, 22]"
0,3,"[11, 22]"
1,foo,[]
2,NaN,[]
3,5,"[99, 88]"
3,4,"[99, 88]"


,A,B
0,1,11
0,1,22
0,2,11
0,2,22
0,3,11
0,3,22
1,foo,NaN
2,NaN,NaN
3,5,99
3,5,88


In [18]:
dfmg.head

<bound method NDFrame.head of            chr        pos ref alt                    patient      impact  \
0            1  121484536   C   T  HTMCP-03-06-02025-01A-01D  INTERGENIC   
1            1  121484671   T   G  HTMCP-03-06-02025-01A-01D  INTERGENIC   
2            1  121484700   T   A  HTMCP-03-06-02025-01A-01D  INTERGENIC   
3            1  121485137   G   C  HTMCP-03-06-02025-01A-01D  INTERGENIC   
4            2   92315416   T   C  HTMCP-03-06-02025-01A-01D  INTERGENIC   
..         ...        ...  ..  ..                        ...         ...   
17          19   27738503   A   C  HTMCP-03-06-02060-01A-01D  INTERGENIC   
18  GL000220.1     118534   T   G  HTMCP-03-06-02060-01A-01D  DOWNSTREAM   
18  GL000220.1     118534   T   G  HTMCP-03-06-02060-01A-01D  DOWNSTREAM   
18  GL000220.1     118534   T   G  HTMCP-03-06-02060-01A-01D  INTERGENIC   
19  GL000220.1     125691   C   G  HTMCP-03-06-02060-01A-01D  INTERGENIC   

   impact_type type AA_change         gene       transcri

In [15]:
dfmg.shape
dfmg.head(2)

dfmg[dfmg.pos == 142706206]

(27, 11)

,chr,pos,ref,alt,patient,impact,impact_type,type,AA_change,gene,transcript
0,1,121484536,C,T,HTMCP-03-06-02025-01A-01D,INTERGENIC,MODIFIER,,,,
1,1,121484671,T,G,HTMCP-03-06-02025-01A-01D,INTERGENIC,MODIFIER,,,,


,chr,pos,ref,alt,patient,impact,impact_type,type,AA_change,gene,transcript
6,6,142706206,G,A,HTMCP-03-06-02025-01A-01D,DOWNSTREAM,MODIFIER,,,GPR126,ENST00000545477
6,6,142706206,G,A,HTMCP-03-06-02025-01A-01D,INTRON,MODIFIER,,,GPR126,ENST00000367609
6,6,142706206,G,A,HTMCP-03-06-02025-01A-01D,INTRON,MODIFIER,,,GPR126,ENST00000230173
6,6,142706206,G,A,HTMCP-03-06-02025-01A-01D,INTRON,MODIFIER,,,GPR126,ENST00000367608
6,6,142706206,G,A,HTMCP-03-06-02025-01A-01D,INTRON,MODIFIER,,,GPR126,ENST00000296932


In [66]:
!python notebook2script.py VcfParser.ipynb

Converted VcfParser.ipynb to exp/nb_VcfParser.py


In [93]:
s = '53:53:52,52:0,0:1,1:53.55:0.00:0.00'
def parse_indel_coverage(s):
    cov = s.split(':')
    cov1 = cov[0]
    alt1 = cov[2].split(',')[0]
#     indel1 = cov[3].split(',')[0]
    print(cov1, alt1)
parse_indel_coverage(s)

53 52


In [119]:
s = '64:3:0:0:50,72:11,15:0,0:0,0'
bases = np.array(['A', 'C', 'G', 'T'])
def parse_snv_coverage(s, alt):
    # DP:FDP:SDP:SUBDP:AU:CU:GU:TU
    cov = s.split(':')
    acgt = np.zeros(4)
    for i, c in enumerate(cov[4:]):
        acgt[i] = c.split(',')[0]
    return [acgt.sum(), acgt[np.asscalar(np.where(bases == alt)[0])]]
parse_snv_coverage(s, 'G')

[61.0, 0.0]

In [117]:
bases[np.where(bases == 'G')[0]]

array(['G'], dtype='<U1')

In [105]:
np.where(bases == 'G')

(array([2]),)

In [109]:
bases[3]

'T'

In [118]:
a = np.zeros(4)
a[1]

0.0

In [25]:
from functools import reduce

primes = [2, 3, 5, 7, 11, 13]

def product(*numbers):
    p = reduce(lambda x, y: x * y, numbers)
    return p 

product(*primes)
# 30030

product(primes)
# [2, 3, 5, 7, 11, 13]


30030

[2, 3, 5, 7, 11, 13]

In [29]:
primes
print(*primes)

[2, 3, 5, 7, 11, 13]

2 3 5 7 11 13
